In [11]:
from bs4 import BeautifulSoup
import pymysql
import requests
# import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_palette('Set3')

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["font.family"] = "NanumBarunGothic"

import re

In [27]:
# data load
# pip install openpyxl
path = "C:/프로젝트/k-d/2-Project/주제/data"
# main = pd.read_excel(path + '/국립중앙도서관 대출 데이터/NL_CO_LOAN_PUB_202311-7.xlsx')
Top_Lib = pd.read_excel(path + '/크롤링select데이터/제목_ISBN중복제거(3).xlsx')


num_rows, num_columns = Top_Lib.shape

# 결과 출력
print(f"데이터 수: {num_rows}")

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

데이터 수: 2


In [9]:
# Test 실행
# #headers = {
#   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
#}

#for index, count in Top_Lib.iterrows():
#    response = requests.get("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(count['ISBN13번호']), headers=headers)
#    URL = ("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(count['ISBN13번호']))
#    print("URL 확인", URL)

In [41]:
conn = pymysql.connect(host='127.0.0.1',
                       user='root',
                       password='1234',
                       db='Test_one',
                       charset='utf8')
cursor = conn.cursor()
headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

Book_count = 0

for index, count in Top_Lib.iterrows():
    response = requests.get("https://search.kyobobook.co.kr/search?keyword={0}&gbCode=TOT&target=total".format(count['ISBN13번호']), headers=headers)
        
    html = response.content.decode('utf-8','replace')
    soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

    my_st = soup.select(
        '#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box'
    )
    
    for stock_name in my_st:
        try:
            price_elements = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_price > span.price > span.val")

            print(price_elements)
            if price_elements == []:
                Price_varchar = -1
            else:
                if price_elements and price_elements[0]:
                    Book_Price = price_elements[0].text
                    Price_varchar = Book_Price.replace(',','')
                    print(Price_varchar)
                    
                else:
                    Book_Price = "0"

        except IndexError as e:
            print("Book_Price 처리 중 에러:", e)
            Book_Price = "0"
        
        try:
            Vente_exposant = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box")
        
            if Vente_exposant and Vente_exposant[0]:
                Vente_exposant_text = Vente_exposant[0].text

                Vente_exposant_text_one = '%-11s' % (' ').join(Vente_exposant_text.split())
                matches = re.findall(r'[\d.]+', Vente_exposant_text_one)
                # print(Vente_exposant_text_one)
                ranking = matches[0]
            else:
                Vente_exposant_text = "0"

        except IndexError as e:
            print("review_desc 처리 중 에러:", e)
            Vente_exposant_text = "0"

        try:
            Vente_exposant = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box")
        
            if Vente_exposant and Vente_exposant[0]:
                Vente_exposant_text = Vente_exposant[0].text

                Vente_exposant_text_one = '%-11s' % (' ').join(Vente_exposant_text.split())
                matches = re.findall(r'[\d.]+', Vente_exposant_text_one)
                # print(Vente_exposant_text_one)
                review_desc = matches[1]
            else:
                Vente_exposant_text = "0"

        except IndexError as e:
            print("review_desc 처리 중 에러:", e)
            Vente_exposant_text = "0"
       
    
    Book_count += 1
        # Replace '무료도서' with 0 for the 'price' column
    sql = "INSERT INTO Test_one.Book_Review (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('{0}', {1}, {2}, {3});".format(str(count['ISBN13번호']),0 if Price_varchar == '무료도서' or Price_varchar == '절판되었습니다.' else int(Price_varchar) if Price_varchar is not None else 0,ranking, review_desc)
    print(Book_count, "/", {num_rows})
    print(sql)
    conn.commit()
    cursor.execute(sql)

[<span class="val">무료도서</span>]
무료도서
1 / {2}
INSERT INTO Test_one.Book_Review (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791189026202', 0, 10.0, 2);
[]
2 / {2}
INSERT INTO Test_one.Book_Review (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791189015657', -1, 9.4, 15);
